In [9]:
import json
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import numpy as np

In [23]:
import voyageai

vo = voyageai.Client(api_key="pa-qePwGAS_4DABZtDIzXKIQdsYkayhv2s770Tsw3XwAHt")
# This will automatically use the environment variable VOYAGE_API_KEY.
# Alternatively, you can use vo = voyageai.Client(api_key="<your secret key>")

result = vo.embed(["hello world"], model="voyage-3-large")

In [10]:
# 1. import testing Question dataset [Q]
with open("../math_qa_all_v1.json", "r") as qf:
    data = json.load(qf)
    page_content = [q['content'] for q in data]
    page_number = [q['page'] for q in data]
    questions: list = [q['Q'] for q in data]
    answers: list = [q['A'] for q in data]

In [21]:
a = []
b = [["b"], ["c"]]
b[0:10]

[['b'], ['c']]

In [24]:
page_embeddings: list = []
questions_embeddings: list = []
for i in range(len(data) // 128 + 1):
    page_embeddings += vo.embed(page_content[i * 128: (i + 1) * 128], model="voyage-3-large").embeddings
    questions_embeddings += vo.embed(questions[i * 128: (i + 1) * 128], model="voyage-3-large").embeddings

# page_embeddings: np.ndarray = model.encode(page_content)
# questions_embeddings: np.ndarray  = model.encode(questions, prompt_name=query_prompt_name)

In [26]:
for i in range(len(data)):
    data[i]['page_embedding'] = page_embeddings[i]
    data[i]['question_embedding'] = questions_embeddings[i]

In [29]:
# question_retrived_page_rank = []
for i in range(len(data)):
    similarity = [float(cos_sim(data[i]['question_embedding'], page_embed)) for page_embed in page_embeddings]
    question_retrived_page_rank = [{k: v} for k, v in zip(page_number, similarity)]
    data[i]['question_retrived_page_rank'] = question_retrived_page_rank

In [30]:
for i in range(len(data)):
    data[i]['question_retrived_page_rank'] = sorted(data[i]['question_retrived_page_rank'], key = lambda d: list(d.values())[0], reverse=True)

In [31]:
top1 = 0
top3 = 0
top5 = 0
top10 = 0
top20 = 0
top50 = 0
top100 = 0

page_distances = [] # the distances of the correct page and retrieved page
scores = [] # the score of the corresponding page and question
correct_rank = [] # the index of the correct page in the retrieved results.

for i in range(len(data)):
    page_number = data[i]['page']
    retrived_page_rank = [list(data[i]['question_retrived_page_rank'][d].keys())[0] for d in range(len(data[i]['question_retrived_page_rank']))]
    correct_page_rank = retrived_page_rank.index(page_number)
    correct_rank.append(correct_page_rank)
    page_distances.append(abs(page_number - retrived_page_rank[0]))
    scores.append(cos_sim(data[i]['question_embedding'], data[i]['page_embedding']))
    
    if page_number in retrived_page_rank[:1]: top1 += 1
    if page_number in retrived_page_rank[:3]: top3 += 1
    if page_number in retrived_page_rank[:5]: top5 += 1
    if page_number in retrived_page_rank[:10]: top10 += 1
    if page_number in retrived_page_rank[:20]: top20 += 1
    if page_number in retrived_page_rank[:50]: top50 += 1
    if page_number in retrived_page_rank[:100]: top100 += 1   

In [32]:
print(top1)
print(top3)
print(top5)
print(top10)
print(top20)
print(top50)
print(top100)


327
434
457
474
474
476
477


In [33]:
print("Avg score: " + str(float(sum(scores) / len(scores))))
print("Avg rank: " + str(sum(correct_rank) / len(correct_rank)))
print("Avg page distance: " + str(sum(page_distances)/len(page_distances)))

Avg score: 0.7667664885520935
Avg rank: 0.8742138364779874
Avg page distance: 12.184486373165619


In [34]:
import pickle

with open('voyage.pkl', 'wb') as f:
    pickle.dump(data, f)
    